In [43]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

unscaled_input_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:,-1]

### Balancing the data

In [44]:
num_one_all = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_all:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_input_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [45]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shffle the data

In [46]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the data in Train, Validation and Test

In [47]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count: train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count: train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets),test_samples_count, np.sum(test_targets)/test_samples_count)

1804.0 3579 0.5040514110086617
207.0 447 0.46308724832214765
226.0 448 0.5044642857142857


In [48]:
np.savez('Audiobook-data-train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobook-data-validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobook-data-test', inputs=test_inputs, targets=test_targets)

# Machine Learnig Algo

In [49]:
import numpy as np
import tensorflow as tf

In [50]:
npz = np.load('Audiobook-data-train.npz')

train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('Audiobook-data-validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('Audiobook-data-test.npz')
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

### Model

In [81]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

#helps in case of overtraining
#patience let us decide how many consicutive increase will be tolerated
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
         train_targets,
         batch_size = batch_size,
         epochs = max_epochs,
         callbacks = [early_stopping],
         validation_data=(validation_inputs, validation_targets),
         verbose=2)

Epoch 1/100
36/36 - 1s - loss: 0.6072 - accuracy: 0.6667 - val_loss: 0.5071 - val_accuracy: 0.7539 - 1s/epoch - 28ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4685 - accuracy: 0.7625 - val_loss: 0.4311 - val_accuracy: 0.7785 - 83ms/epoch - 2ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4178 - accuracy: 0.7762 - val_loss: 0.3971 - val_accuracy: 0.7808 - 81ms/epoch - 2ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3926 - accuracy: 0.7832 - val_loss: 0.3760 - val_accuracy: 0.8076 - 83ms/epoch - 2ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3786 - accuracy: 0.7977 - val_loss: 0.3641 - val_accuracy: 0.8188 - 83ms/epoch - 2ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3702 - accuracy: 0.7988 - val_loss: 0.3615 - val_accuracy: 0.8166 - 99ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3608 - accuracy: 0.8025 - val_loss: 0.3549 - val_accuracy: 0.8233 - 120ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3556 - accuracy: 0.8125 - val_loss: 0.3437 - val_accuracy: 0.8210 - 139ms/epoch - 4ms/step
Epoch 9/100
36/

### Test the model

In [82]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3361 - accuracy: 0.8214


In [83]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'. format(test_loss, test_accuracy*100.))


Test loss: 0.34. Test accuracy: 82.14%
